<a href="https://colab.research.google.com/github/kimgm1018/G_every_time_clowl/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertForMultipleChoice, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/한국어데이터정보학_논문/clow_data_final.xlsx")
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

,time,URL,title,article,reple
0,2024/10/31,https://everytime.kr/380952/v/358786915,미니게이야... 이거 보여주려고 챗지피티 결제했다...,대 국립경상대학교의 이름을 빚네는 너희들이 자랑스럽다!!!!,ㅋㅋㅋㅋㅋㅋㅋ\nㅣ발 도서관에서 현웃터짐 ㅋㅋㅋㅋ\nㅈㄴ웃기네 ㅋㅋㅋㅋㅋㅋ\nㅋㅋㅋ...
1,2024/10/31,https://everytime.kr/380952/v/358776030,담배값 7000원 인상 뭐야??,담배값 7000원 기사 링크 이거 되면 담배 어떻게 피냐…...... 경상국립대 학...,흡연충 = 똥 아무데나 싸재끼는 똥개\n흡연장 가서 담배피는 사람들도 있습니다ㅠㅠ\...
2,2024/10/31,https://everytime.kr/380952/v/358767790,실시간,빨간 스팅어 국제어학원앞 등장!!!,나도 봤다!!!\n아 또못봤네 하 졸업전에 한번 보고싶다
3,2024/10/31,https://everytime.kr/380952/v/358768087,💀그 귀신 접니다👻,친구들과 사진찍고 놀기위해 입고 왔습니다..할로윈을 즐겨서 행복합니다...민폐였다면...,붕어빵 이모님도 즐거워하셨습니다\n귀엽다ㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n컨셉 진짜 ㅋㅋㅋㅋㅋㅋㅋ...
4,2024/10/31,https://everytime.kr/380952/v/358747507,🎃경상대 귀신 출몰🎃,할로윈을 밎아서 경상대에 귀신이 등장했어요!,ㅅㅂ 난 코스프레 안해도 귀신 닮았다고 하던데\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n할로윈 추ㅋㅋ...
...,...,...,...,...,...
403,2024/09/02,https://everytime.kr/380952/v/353119360,학교 주차장 건의사항(핫게 보내주세요),자차로 다른 지역에서 통학하는 학생입니다제가 오늘 느꼈던 불편하고 개선했으면 좋겠는...,교내 주차공간이 협소해서 학생들까지 차 들고 오면 도로도 좁고 안전사고도 생길 수 ...
404,2024/09/02,https://everytime.kr/380952/v/353092326,종강할때 된거같은데,왜 종강안하지,12월 20일까지 어케 버티뉴..ㅠㅠ\n엥 종강일 왤케 한참 남음 ㄷㄷ;;\nㅇㅈ
405,2024/09/02,https://everytime.kr/380952/v/353064893,요즘 내 학교생활 요약해봄.jpg,.,ㅅㅌㅊ\n하교짤 심지어 그냥 등교짤 좌우반전한거 ㅋㅋㅋ\n좌우반전이 웃기네 ㅋㅋㅋ\...
406,2024/09/01,https://everytime.kr/380952/v/352985745,선배랑 말싸움하다가 지 주제도 모른다고 하길래,토지를 소작농에게 빌려주고 대신 지대를 받는 제도라고 또박또박 대답함,농평 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n지주제 설명은 역시 농평 ㅋㅋ\nㅋㅋㅋ\n경상대 근본 ㅋㅋ...


In [ ]:
df['reple']

,reple
0,ㅋㅋㅋㅋㅋㅋㅋ\nㅣ발 도서관에서 현웃터짐 ㅋㅋㅋㅋ\nㅈㄴ웃기네 ㅋㅋㅋㅋㅋㅋ\nㅋㅋㅋ...
1,흡연충 = 똥 아무데나 싸재끼는 똥개\n흡연장 가서 담배피는 사람들도 있습니다ㅠㅠ\...
2,나도 봤다!!!\n아 또못봤네 하 졸업전에 한번 보고싶다
3,붕어빵 이모님도 즐거워하셨습니다\n귀엽다ㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n컨셉 진짜 ㅋㅋㅋㅋㅋㅋㅋ...
4,ㅅㅂ 난 코스프레 안해도 귀신 닮았다고 하던데\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n할로윈 추ㅋㅋ...
...,...
403,교내 주차공간이 협소해서 학생들까지 차 들고 오면 도로도 좁고 안전사고도 생길 수 ...
404,12월 20일까지 어케 버티뉴..ㅠㅠ\n엥 종강일 왤케 한참 남음 ㄷㄷ;;\nㅇㅈ
405,ㅅㅌㅊ\n하교짤 심지어 그냥 등교짤 좌우반전한거 ㅋㅋㅋ\n좌우반전이 웃기네 ㅋㅋㅋ\...
406,농평 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n지주제 설명은 역시 농평 ㅋㅋ\nㅋㅋㅋ\n경상대 근본 ㅋㅋ...


In [ ]:

# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 135

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks



# 문장 테스트
def test_sentences(model, sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits


In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

SystemError: GPU device not found

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [ ]:
# for i in df['reple'].loc[10]:
#   print(i)

print(df['reple'].loc[1])

흡연충 = 똥 아무데나 싸재끼는 똥개
흡연장 가서 담배피는 사람들도 있습니다ㅠㅠ
라는 말 듣기 싫으니 제발 궐련 피운거는 잘 치웁시다 길빵 자제하시구요 제발..
꽁초까지는 하도 많이 보여서 그려러니 하는데 담배 갑 버리는건 도대체 뭐임 진짜
화단에 버린건 나무 안 밟게 조심히 걸어가서 주웠답니다
한국인의 시민의식은 개선될 수 있는게 아니다싫으면 중이 떠나자한국에 살거면 들이마시라잉~
절이 낡았으면 수리해야죠, 떠날 게 아니라.
흡연충 이 미친새기들
투기장 열리면 게시글 삭제합니다
삭제된 댓글입니다.
그건 올바른 방법이 아닙니다. 아무 곳이나 버리는 행동에 대해 스스로 경계하는게 아닌 그저 처벌에 대한 두려움으로 경계하게 된다면 일시적이고 더 숨어들어갈 뿐입니다. 징벌적 행동변화가 아닌 인도적 행동변화를 이끌어야 선한 마음으로 선한 행동을 할 수 있습니다.
청소하면서 담배를 올바르게 버리는 사람과 그저 그대로 버리는 사람이 있었습니다. 저의 방식은 오래 걸리겠지만 확실한 방법이라 생각하여 지난 시험기간동안 치운거에요
일단 난 고소먹기 싫어서 무서우니까 님이 해주셈
멋있다 ㄷㄷ
당신 산시공입니까…?
어..어떻게 아셨나요 산시공 최고 미남입니다
만점 드립니다.내일 생산운영관리 파이팅..
🚬🧹👍💯
와 흡연자를 좋게 생각한 적은 없었는데 진짜 멋있다..
오지랖은 ㅋㅋ
베베 꼬였긴 ㅋㅋ
ㅈㄴ 꼬였노
진짜 착하다....
오늘 샤프심 고맙다!


In [ ]:
# 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/한국어데이터정보학_논문/_model1/fine-tuned-model', do_lower_case = False)
model1 = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/한국어데이터정보학_논문/_model1/fine-tuned-model')

In [ ]:
def logit_to_label(logits):

  answer = []

  for i in logits:
    answer2 = []
    for j in i:
      if float(tf.math.sigmoid(j)) >= 0.5:
        answer2.append(1)
      else:
        answer2.append(0)

    answer.append(answer2)

  return answer

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(df['reple']):
  sen = i.split('\n')
  logits = test_sentences(model1, sen)
  print(logit_to_label(logits))
  break

  0%|          | 0/408 [00:07<?, ?it/s]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [ ]:
logits = test_sentences(model1, ['흡연충'])

print(tf.math.sigmoid(logits[0]))

answer = []

for i in logits:
  for j in i:
    if float(tf.math.sigmoid(j)) >= 0.3:
      answer.append(1)
    else:
      answer.append(0)

print(answer)

tf.Tensor(
[0.03569712 0.02363395 0.02874905 0.03901098 0.02574502 0.02449714
 0.00778501 0.06457447 0.6872626  0.17138839 0.02843808], shape=(11,), dtype=float32)
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


## 스마일 게이트가 만들어 놓은 model 사용

In [ ]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer

model_name = 'smilegate-ai/kor_unsmile'

model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=-1,     # cpu: -1, gpu: gpu number
    return_all_scores=True,
    function_to_apply='sigmoid'
    )

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/721k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
for result in pipe('무토바'):
  for r in result:
    print(r)

{'label': '여성/가족', 'score': 0.009689804166555405}
{'label': '남성', 'score': 0.009172235615551472}
{'label': '성소수자', 'score': 0.008818679489195347}
{'label': '인종/국적', 'score': 0.02845444716513157}
{'label': '연령', 'score': 0.008513721637427807}
{'label': '지역', 'score': 0.020189383998513222}
{'label': '종교', 'score': 0.011564542539417744}
{'label': '기타 혐오', 'score': 0.00695412140339613}
{'label': '악플/욕설', 'score': 0.08272047340869904}
{'label': 'clean', 'score': 0.8401299118995667}


In [ ]:
def total_argmax(siries):

  total = []

  for s in siries:

    if len(str(s)) > 300:
      s = s[:300]

    sent = str(s).split('\n')

    sent_list = []

    for s in sent:
      for result in pipe(s):
        answer = []
        for r in result:
          answer.append(r['score'])
        sent_list.append(result[np.argmax(answer)]['label'])
    total.append(sent_list)

  return total

In [ ]:
df_title = pd.DataFrame({'title_eval' : []})

# df_title = total_argmax(df_title, df['title'])


df_title

,title_eval


In [ ]:
# df = pd.DataFrame(['title_eval'])

df_title['title_eval'] = total_argmax(df['title'])

df_title

,title_eval
0,[clean]
1,[clean]
2,[clean]
3,[clean]
4,[clean]
...,...
403,[clean]
404,[clean]
405,[clean]
406,[clean]


In [ ]:
df_article = pd.DataFrame({'article_eval' : []})

df_article

,article_eval


In [ ]:
df_article['article_eval'] = total_argmax(df['article'])

df_article

,article_eval
0,[clean]
1,[clean]
2,[clean]
3,[clean]
4,[clean]
...,...
403,[clean]
404,[clean]
405,[clean]
406,[clean]


In [ ]:
df_total_eval = pd.concat([df_title, df_article], axis=1)

df_total_eval

,title_eval,article_eval
0,[clean],[clean]
1,[clean],[clean]
2,[clean],[clean]
3,[clean],[clean]
4,[clean],[clean]
...,...,...
403,[clean],[clean]
404,[clean],[clean]
405,[clean],[clean]
406,[clean],[clean]


In [ ]:
df_reple = pd.DataFrame({'reple_eval' : []})

df_reple

,reple_eval


In [ ]:
df_reple['reple_eval'] = total_argmax(df['reple'])

df_reple

,reple_eval
0,"[clean, clean, clean, clean, clean, clean, cle..."
1,"[연령, clean, clean, clean, clean, 인종/국적, clean,..."
2,"[clean, clean]"
3,"[clean, clean, clean, clean, clean, clean, cle..."
4,"[악플/욕설, clean, clean, 악플/욕설, clean, clean, cle..."
...,...
403,"[clean, clean, clean, clean]"
404,"[clean, clean, clean]"
405,"[clean, clean, clean, clean, clean, 악플/욕설, cle..."
406,"[clean, clean, clean, 지역, clean, clean, clean,..."


In [ ]:
df_total_eval = pd.concat([df_total_eval, df_reple], axis=1)

df_total_eval

,title_eval,article_eval,reple_eval
0,[clean],[clean],"[clean, clean, clean, clean, clean, clean, cle..."
1,[clean],[clean],"[연령, clean, clean, clean, clean, 인종/국적, clean,..."
2,[clean],[clean],"[clean, clean]"
3,[clean],[clean],"[clean, clean, clean, clean, clean, clean, cle..."
4,[clean],[clean],"[악플/욕설, clean, clean, 악플/욕설, clean, clean, cle..."
...,...,...,...
403,[clean],[clean],"[clean, clean, clean, clean]"
404,[clean],[clean],"[clean, clean, clean]"
405,[clean],[clean],"[clean, clean, clean, clean, clean, 악플/욕설, cle..."
406,[clean],[clean],"[clean, clean, clean, 지역, clean, clean, clean,..."


In [ ]:
df_total_eval.to_excel("/content/drive/MyDrive/Colab Notebooks/한국어데이터정보학_논문/크롤링 data/total_eval.xlsx")

NameError: name 'df_total_eval' is not defined

In [ ]:
for reple in df['title']:
  re = reple.split('\n')
  for k in re:
    print(k)
    for result in pipe(k):
      for r in result:
        print(r)
      print("=======================================================")

ㅋㅋㅋㅋㅋㅋㅋ
{'label': '여성/가족', 'score': 0.01344284787774086}
{'label': '남성', 'score': 0.010367188602685928}
{'label': '성소수자', 'score': 0.01132824458181858}
{'label': '인종/국적', 'score': 0.013440857641398907}
{'label': '연령', 'score': 0.00886013824492693}
{'label': '지역', 'score': 0.01500460971146822}
{'label': '종교', 'score': 0.01246666070073843}
{'label': '기타 혐오', 'score': 0.007809486240148544}
{'label': '악플/욕설', 'score': 0.08928833901882172}
{'label': 'clean', 'score': 0.8848084807395935}
ㅣ발 도서관에서 현웃터짐 ㅋㅋㅋㅋ
{'label': '여성/가족', 'score': 0.014346358366310596}
{'label': '남성', 'score': 0.010057498700916767}
{'label': '성소수자', 'score': 0.009655938483774662}
{'label': '인종/국적', 'score': 0.013244827277958393}
{'label': '연령', 'score': 0.0082972701638937}
{'label': '지역', 'score': 0.014135858044028282}
{'label': '종교', 'score': 0.008740631863474846}
{'label': '기타 혐오', 'score': 0.007802695035934448}
{'label': '악플/욕설', 'score': 0.12067296355962753}
{'label': 'clean', 'score': 0.8531430959701538}
ㅈㄴ웃기네 ㅋㅋㅋㅋㅋㅋ

KeyboardInterrupt: 

In [ ]:
kkkk = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/한국어데이터정보학_논문/total_eval.xlsx')

kkkk.drop(['Unnamed: 0'], axis=1, inplace=True)

kkkk

,title_eval,article_eval,reple_eval
0,['clean'],['clean'],"['clean', 'clean', 'clean', 'clean', 'clean', ..."
1,['clean'],['clean'],"['연령', 'clean', 'clean', 'clean', 'clean', '인종..."
2,['clean'],['clean'],"['clean', 'clean']"
3,['clean'],['clean'],"['clean', 'clean', 'clean', 'clean', 'clean', ..."
4,['clean'],['clean'],"['악플/욕설', 'clean', 'clean', '악플/욕설', 'clean', ..."
...,...,...,...
403,['clean'],['clean'],"['clean', 'clean', 'clean', 'clean']"
404,['clean'],['clean'],"['clean', 'clean', 'clean']"
405,['clean'],['clean'],"['clean', 'clean', 'clean', 'clean', 'clean', ..."
406,['clean'],['clean'],"['clean', 'clean', 'clean', '지역', 'clean', 'cl..."


In [ ]:
title_hate = 0
article_hate = 0
hate_reple = {0 : [], 1 : [], 2 : []}

for i in range(len(kkkk['title_eval'])):
  if 'clean' in  kkkk.loc[i]['title_eval']:
    if 'clean' in  kkkk.loc[i]['article_eval']:

      hate_reple[0].append(len(kkkk.loc[i]['reple_eval'].split(',')))

    else:
      hate_reple[1].append(len(kkkk.loc[i]['reple_eval'].split(',')))

  else:
    if 'clean' in  kkkk.loc[i]['article_eval']:
      hate_reple[1].append(len(kkkk.loc[i]['reple_eval'].split(',')))
    else:
      hate_reple[2].append(len(kkkk.loc[i]['reple_eval'].split(',')))


In [ ]:
hate_reple

print(sum(hate_reple[0]) / len(hate_reple[0]), len(hate_reple[0]))
print(max(hate_reple[0]), min(hate_reple[0]))
print(sum(hate_reple[1]) / len(hate_reple[1]), len(hate_reple[1]))
print(max(hate_reple[1]) , min(hate_reple[0]))
print(sum(hate_reple[2]) / len(hate_reple[2]), len(hate_reple[2]))
print(max(hate_reple[2]) , min(hate_reple[0]))

9.880701754385965 285
28 1
10.135416666666666 96
26 1
9.666666666666666 27
19 1


In [ ]:
# 최대 댓글 수 확인

# print(stats.trim_mean(hate_reple[0], 0.2))
print(max(hate_reple[0]))
abcd  = []
abcd.extend(hate_reple[1])
abcd.extend(hate_reple[2])
print(max(abcd))
# print(stats.trim_mean(abcd, 0.2))
# print(stats.trim_mean(hate_reple[1], 0.2))
# print(stats.trim_mean(hate_reple[2], 0.2))

28
26


In [ ]:
from scipy import stats

In [ ]:
reple_num = []

for i in kkkk['reple_eval']:
  reple_num.append(len(i.split(',')))

In [ ]:
# 전체 댓글 평균 확인

print(reple_num)
print(sum(reple_num), sum(reple_num) / len(reple_num))
print(stats.trim_mean(reple_num, 0.2))

[18, 11, 2, 19, 15, 15, 1, 4, 10, 24, 16, 2, 4, 8, 10, 15, 24, 9, 10, 9, 24, 6, 14, 17, 10, 9, 21, 12, 12, 4, 8, 14, 22, 12, 14, 9, 8, 7, 18, 1, 10, 10, 13, 6, 11, 5, 13, 10, 7, 10, 6, 2, 10, 3, 8, 11, 9, 11, 12, 4, 3, 5, 5, 11, 7, 6, 8, 13, 3, 6, 4, 10, 13, 7, 11, 7, 10, 11, 4, 4, 6, 9, 14, 8, 6, 19, 1, 4, 1, 6, 4, 14, 13, 6, 6, 9, 7, 14, 11, 7, 7, 8, 4, 5, 18, 13, 4, 10, 8, 3, 1, 14, 5, 9, 6, 8, 11, 3, 8, 7, 9, 8, 7, 12, 1, 14, 9, 8, 10, 2, 10, 4, 6, 1, 7, 13, 13, 20, 2, 2, 7, 14, 6, 15, 11, 9, 3, 4, 7, 4, 5, 16, 12, 10, 9, 19, 1, 5, 13, 9, 13, 12, 8, 8, 16, 2, 17, 7, 13, 13, 7, 7, 8, 6, 19, 9, 19, 13, 4, 13, 12, 19, 12, 10, 18, 14, 14, 14, 13, 3, 15, 14, 13, 14, 8, 9, 2, 9, 8, 9, 7, 11, 4, 11, 4, 9, 2, 9, 8, 4, 14, 15, 13, 3, 8, 9, 12, 13, 11, 5, 17, 14, 1, 3, 11, 2, 8, 2, 15, 9, 6, 2, 3, 2, 8, 12, 11, 10, 11, 20, 20, 9, 14, 11, 5, 18, 6, 19, 19, 4, 21, 14, 1, 12, 23, 14, 10, 11, 11, 1, 8, 5, 25, 8, 18, 10, 9, 11, 5, 16, 15, 15, 19, 19, 13, 13, 10, 13, 10, 5, 11, 10, 6, 26, 9, 22, 1

> 각 혐오표현 별 댓글 평균 확인

In [ ]:
gender_reple = []

for i in range(len(kkkk['title_eval'])):
  if '남성' in kkkk.loc[i]['title_eval'] or '여성' in kkkk.loc[i]['title_eval'] or '남성' in kkkk.loc[i]['article_eval'] or '여성' in kkkk.loc[i]['article_eval'] or '남성' in kkkk.loc[i]['reple_eval'] or '여성' in kkkk.loc[i]['reple_eval']:
    gender_reple.append(len(kkkk.loc[i]['reple_eval'].split(',')))


In [ ]:
print(gender_reple)
print(sum(gender_reple) / len(gender_reple))
print(stats.trim_mean(gender_reple, 0.2))

[8, 3, 10, 4, 8, 14, 8, 7, 12, 16, 1, 17, 12, 18, 13, 9, 11, 5, 1, 9, 13, 5, 26, 14, 18, 18, 15, 9, 3, 8, 15, 16, 16, 8, 8, 5, 7, 12, 10, 12, 13, 8]
10.595238095238095
10.5


In [ ]:
fem_reple = []

for i in range(len(kkkk['title_eval'])):
  if '여성' in kkkk.loc[i]['title_eval'] or '여성' in kkkk.loc[i]['article_eval'] or '여성' in kkkk.loc[i]['reple_eval']:
    fem_reple.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(fem_reple, 0.2))

9.764705882352942


In [ ]:
m_reple = []

for i in range(len(kkkk['title_eval'])):
  if '남성' in kkkk.loc[i]['title_eval'] or '남성' in kkkk.loc[i]['article_eval'] or '남성' in kkkk.loc[i]['reple_eval']:
    m_reple.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(m_reple, 0.2))

11.76923076923077


In [ ]:
old_reple = []

for i in range(len(kkkk['title_eval'])):
  if '연령' in kkkk.loc[i]['title_eval'] or '연령' in kkkk.loc[i]['article_eval'] or '연령' in kkkk.loc[i]['reple_eval']:
    old_reple.append(len(kkkk.loc[i]['reple_eval'].split(',')))

In [ ]:
print(sum(old_reple), sum(old_reple) / len(old_reple))
print(stats.trim_mean(old_reple, 0.2))

163 13.583333333333334
12.375


In [ ]:
j = []

for i in range(len(kkkk['title_eval'])):
  if '지역' in kkkk.loc[i]['title_eval'] or '지역' in kkkk.loc[i]['article_eval'] or '지역' in kkkk.loc[i]['reple_eval']:
    j.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(j, 0.2))

12.538461538461538


In [ ]:
sosu = []

for i in range(len(kkkk['title_eval'])):
  if '성소수자' in kkkk.loc[i]['title_eval'] or '성소수자' in kkkk.loc[i]['article_eval'] or '성소수자' in kkkk.loc[i]['reple_eval']:
    sosu.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(sosu, 0.2))

11.0


In [ ]:
ack = []

for i in range(len(kkkk['title_eval'])):
  if '악플/욕설' in kkkk.loc[i]['title_eval'] or '악플/욕설' in kkkk.loc[i]['article_eval'] or '악플/욕설' in kkkk.loc[i]['reple_eval']:
    ack.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(ack, 0.2))

10.709219858156029


In [ ]:
ost = []

for i in range(len(kkkk['title_eval'])):
  if '인종/국적' in kkkk.loc[i]['title_eval'] or '인종/국적' in kkkk.loc[i]['article_eval'] or '인종/국적' in kkkk.loc[i]['reple_eval']:
    ost.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(ost, 0.2))

12.666666666666666


In [ ]:
else_reple = []

for i in range(len(kkkk['title_eval'])):
  if '기타 혐오' in kkkk.loc[i]['title_eval'] or '기타 혐오' in kkkk.loc[i]['article_eval'] or '기타 혐오' in kkkk.loc[i]['reple_eval']:
    else_reple.append(len(kkkk.loc[i]['reple_eval'].split(',')))

print(stats.trim_mean(else_reple, 0.2))

11.833333333333334


In [ ]:
from collections import Counter

In [ ]:
for i in kkkk['reple_eval']:
  print(i[1:-2].split(', '))
  print(Counter((i[1:-2].split(', '))))
  break

["'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'clean'", "'악플/욕설'", "'clean'", "'clean'", "'clean"]
Counter({"'clean'": 16, "'악플/욕설'": 1, "'clean": 1})


-------------------------

In [ ]:
from collections import Counter

In [ ]:
def count_label(pd_series):

  kkk = []

  for i in pd_series:
    for j in i:
      kkk.append(j)

  return Counter(kkk)

In [ ]:
print(count_label(df_total_eval['title_eval']))
print(count_label(df_total_eval['article_eval']))
print(count_label(df_total_eval['reple_eval']))

Counter({'clean': 353, '악플/욕설': 44, '지역': 4, '여성/가족': 3, '성소수자': 1, '연령': 1, '남성': 1, '종교': 1})
Counter({'clean': 313, '악플/욕설': 70, '여성/가족': 7, '지역': 6, '연령': 4, '남성': 3, '성소수자': 2, '인종/국적': 2, '기타 혐오': 1})
Counter({'clean': 3526, '악플/욕설': 428, '여성/가족': 27, '남성': 19, '지역': 14, '기타 혐오': 11, '연령': 10, '성소수자': 8, '인종/국적': 7})


In [ ]:
print(len(df_total_eval['title_eval']))

408
